<a href="https://colab.research.google.com/github/akalpit23/Interpretable-ML/blob/main/Assignment_04_Interpretable_ML_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AIPI 590 - XAI | Assignment #04



## Akalpit Dawkhar

### **Interpretable ML-2**

### Overview

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import lime
import sklearn
from sklearn.metrics import confusion_matrix, roc_curve, auc, accuracy_score, precision_recall_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from imodels import  RuleFitRegressor
from rulefit import RuleFit
from skrules import SkopeRulesClassifier

ImportError: cannot import name 'Iterable' from 'collections' (/Users/akalpitdawkhar/anaconda3/lib/python3.11/collections/__init__.py)

In [35]:
# Load the dataset
credit_card_data = pd.read_csv('creditcard.csv')[:1000]

# Print the shape of the dataset
print(credit_card_data.shape)

# Print the first few rows of the dataset
print(credit_card_data.head())


# Split your data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

(1000, 31)
   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V27

/Users/akalpitdawkhar/anaconda3/lib/python3.11/site-packages/pandas/io/parsers/c_parser_wrapper.py:373: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common_type = np.find_common_type(


In [36]:
# Check the shape of your DataFrame
print(X_train.shape)

# Check the column names of your DataFrame
print(X_train.columns)

(227845, 30)
Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount'],
      dtype='object')


In [37]:
# Load your data
X = credit_card_data.drop('Class', axis=1)  # assuming 'Class' is your target variable
y = credit_card_data['Class']

# Initialize the RuleFitClassifier
rf = RuleFitRegressor()

# Fit the model to your data
rf.fit(X_train, y_train)

# Use the model to make predictions
predictions = rf.predict(X_test)

In [32]:
rule_df = rf.visualize()
rule_df

,rule,coef
1,V1,-0.00
11,V11,0.00
2,V2,0.00
19,V19,0.00
18,V18,-0.00
17,V17,-0.00
16,V16,-0.00
14,V14,-0.00
12,V12,-0.00
21,V21,0.00


In [30]:
# Calculate the mean absolute error
mae = mean_absolute_error(y_test, predictions)

# Calculate the mean squared error
mse = mean_squared_error(y_test, predictions)

# Calculate the R-squared score
r2 = r2_score(y_test, predictions)

# Print the metrics
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R2 Score: {r2}')

MAE: 0.002765304695129495
MSE: 0.000635240249565219
R2 Score: 0.6301335109327806


In [38]:
clf = SkopeRulesClassifier(max_depth_duplication=None,
                 n_estimators=30,
                 precision_min=0.2,
                 recall_min=0.01,
                 feature_names=dataset.feature_names)

X, y = credit_card_data.data, credit_card_data.target > 25
X_train, y_train = X[:len(y)//2], y[:len(y)//2]
X_test, y_test = X[len(y)//2:], y[len(y)//2:]
clf.fit(X_train, y_train)
y_score = clf._score_top_rules(X_test) # Get a risk score for each test example
precision, recall, _ = precision_recall_curve(y_test, y_score)
plt.plot(recall, precision)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision Recall curve')
plt.show()


NameError: name 'SkopeRulesClassifier' is not defined